# 3 Run a simulation

The Jupyter Notebooks in the repository provide an introduction for meshing, simulating with FEM, and visualizing RFA models. The Notebooks can be viewed directly in nbviewer. To execute any of the notebooks either locally or in Google Colaboratory, please see part 0 for setup instructions.

<p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/02.01-Pose-Basics.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>



## 3.1 Code

### 3.1.1 Installing in Colab and loading the packages

First, we install the FEniCS packages for using in Colab

In [ ]:
!apt-get install software-properties-common -y
!add-apt-repository ppa:fenics-packages/fenics -y 
!apt-get update -y
!apt-get install --no-install-recommends fenics -y

First, we load all the necessary packages

In [ ]:
from __future__ import print_function

import numpy as np
import time as tm
from scipy import constants as S
from fenics import *
import sys
import os
import matplotlib.pyplot as plt
pi = S.pi

### 3.1.2 Parameters

The classes shown bellow allow to load the electric, thermal, y cell death parameters.

In [ ]:
class EM_parameters:
    cond = 0.333
    cond_rate = 0.
    V0 = 0. # ground pad voltage
    Vprobe = 100. # probe voltage
    cond_model = 'linear'#'constant' # dependence of electrical conductivity
    cond_vap = 0.5 # conductivity after vapourisation
    imp_max = 120 # impedance threshold for control system
    imp_t_off = 15 # time to switch off probe after imp_max exceeded
    mms_source = Constant(0.) # source term for use in mms
    
class thermal_parameters:
    #restrict_th_mesh = 1  # region to restrict thermal solution to
    #qmet = Constant(0.) # metabolic heat term
    rho_c_t = 1060.*3411. # rho*c in tissue (phase)
    rho_c_v = 4.4e5  # rho*c in vapourised tissue (phase) 
    Tu = 374. # upper transition temp
    Tl = 372. # lower transition temp
    k = Constant(0.56) # thermal conductivity 
    dk = 0. # rate of change of thermal conductivity
    omega = Constant(0.004) # blood perfusion
    rho = Constant(1020.) # density blood
    c = Constant(3640.) # specific heat blood
    T0 = Constant(310.15) # baseline temperature
    T_initial = Constant(310.15) # initial flat temperature profile
    perf_model = 'stop'#
    stop_on_me = True # error given when T change over dt too large
    Q = Constant(0.) # allow custom heat source to be given
    cda_update = True # compute cell death
    p_stop = 1. # value of viability at which to stop perfusion
    nu = 1. # number of iterations of heat solver before updating heat source

class cell_death_parameters:
    A = 7.39e39
    R = S.R
    deltaE = 2.577e5


The next class allows to define the parameters for each domain.

In [ ]:
class MATERIAL(UserExpression):
        def __init__(self, subdomains,s_1,s_2,s_3, s_4,**kwargs):
            super().__init__(**kwargs)
            self.subdomains = subdomains
            self.s_1 = s_1
            self.s_2 = s_2        
            self.s_3 = s_3
            self.s_4 = s_4        
        def eval_cell(self, values, x, cell):
            if self.subdomains[cell.index] == 1:
                values[0] = self.s_1#Atrial wall
            elif self.subdomains[cell.index] == 2:
                values[0] = self.s_2#Connective tissue
            elif self.subdomains[cell.index] == 3:
                values[0] = self.s_3#Plastic
            else:
                values[0] = self.s_4#Blood

### The functions

We briefly describe the functions: *electric_problem* and *rfa_bioheat_problem* commented in the sections 1 and 2. First, the function to solve the electric problem.

In [ ]:
#Function to compute de electric problem
def electric_problem(problemname, mesh, interior, boundaries, emp, Theta, thp):
    
    print("--+--+-- compute RFA SAR --+--+--")
    # set solver params in advance
    #solver = KrylovSolver("cg","jacobi")
    #solver = KrylovSolver("cg", "hypre_euclid")
 
    W = FunctionSpace(mesh, 'CG', 2)
    W_dg = FunctionSpace(mesh, 'DG', 2)
    W_dg0 = FunctionSpace(mesh, 'DG', 0) 

    # interpolate T onto piecewise constant to stabilise conductivity
    T_p = project_axisym(Theta,W_dg0)
    
    r = Expression('x[0]',degree=2)#Define the radius for axisymetric problem

    # set measure
    dss = ds(subdomain_data=boundaries)
    
    
    # symmetry and insulating are natural BCs
    bcs = []
    bcs.append(DirichletBC(W, emp.Vprobe, boundaries, 2))#Active electrode
    bcs.append(DirichletBC(W, emp.Vprobe, boundaries, 5))#Active electrode at 40 Deg
    bcs.append(DirichletBC(W, emp.V0, boundaries, 3))#Passive electrode

    # define variational problem
    U = TrialFunction(W)
    V = TestFunction(W)
   
    # define electrical conductivity depending on temperature
    dependenceTsigma = (1.0 + emp.cond_rate*(T_p))
    
    #Diferent baseline conductivities
    conductivities = [0.28, 0.2, 1e-5 , 0.667]   
    s_1 = conductivities[0] #Atrial wall
    s_2 = conductivities[1] #Connective tissue
    s_3 = conductivities[2] #Plastic
    s_4 = conductivities[3] #Blood
    
    SIGMA = MATERIAL(interior,s_1,s_2,s_3, s_4,degree=0)
    
    a = dependenceTsigma*SIGMA*dot(grad(U), grad(V))*r*dx
    L = emp.mms_source*V*r*dx

    U = Function(W)

    solve(a == L, U, bcs,solver_parameters={'linear_solver':'gmres','preconditioner':'ilu'})


    v = TestFunction(W_dg)
    qRF = TrialFunction(W_dg)
    a = qRF*v*r*dx
    L = v*dependenceTsigma*SIGMA*inner(nabla_grad(U), nabla_grad(U))*r*dx
    qRF = Function(W_dg)
    solve(a == L, qRF,solver_parameters={'linear_solver':'gmres','preconditioner':'ilu'})

    power = assemble(qRF*r*dx)*2.*np.pi
    resistance = (emp.Vprobe)**2/power
    
    print('Resistance: ',resistance)
    print('Voltage: ',emp.Vprobe)
    print('Power: ',power)

    return qRF, resistance, power, U

The function to solve the bioheat problem

In [ ]:

def rfa_bioheat_problem(mesh, interior, boundaries, problemname, dt,tmax, thp, emp):
    
    eps = np.finfo(float).eps # useful quantity
    
    #Parameters for control of temperature
    Theta_target = 353.15-310.15 #80°C 
    xik1 = 0.0 
    Vmax = 66.33
    Vmin = 12.85
    Kp = 4.78 
    Ki = 3.89
    ht = 610.0 #convective parameter
    hp = 3446.0 #convective parameter

    print("--+--+-- Start solving the bioheat equation --+--+--")
  
    # set solver params in advance
    #solver = KrylovSolver("cg","jacobi")
    #solver = KrylovSolver("cg", "hypre_euclid")
   
    W = FunctionSpace(mesh, 'CG', 2)
    W_dg = FunctionSpace(mesh, 'DG', 2) 
    W_dg_0 = FunctionSpace(mesh, 'DG', 0) 
    
    r = Expression('x[0]',degree=2)#Define the radius for axisymetric problem
    
    # set measure
    dss = ds(subdomain_data=boundaries)
    
    # define quantities that need updating
    dte = Expression('dt', degree=1, dt=0.)
    cur_time = Expression('t', degree=1,t=0.)

    # initial uniform temperature
    Theta_prev = project_axisym(thp.T_initial-Constant(310.15),W)
    
    # initial values (if needed)
    resistance = 0.
    
    print("--+--+-- start to solve the electric problem                     --+--+--")
    
    Q, resistance, power, Vfield = electric_problem(problemname, mesh, interior, boundaries, emp, Theta_prev, thp) 
    
    
    qext = project_axisym(Q,W)

    #Apply boundary conditions according to mesh function
    bcs = []
    bcs.append(DirichletBC(W, 0., boundaries, 3))#Passive
    bcs.append(DirichletBC(W, 0., boundaries, 4))#Rest of the model
    bcs.append(DirichletBC(W, Constant(313.15)-thp.T_initial, boundaries, 5))#Constant temperature (40 Deg)
    
    #Define variables for variational form
    Theta_ = TrialFunction(W)
    v = TestFunction(W)
    f = qext 
    
    #vtkfile_calor = File('salida/calor.pvd')
    #vtkfile_calor << project_axisym(qext,W_dg_0)  
       
    #Definition of thermal conductivity
    dependenceTkappa = (1.0 + 0.02*(Theta_prev))    
    kappas = [0.56,0.39,0.026,0.54]   
    s_1 = kappas[0] #Atrial wall
    s_2 = kappas[1] #Connective tissue
    s_3 = kappas[2] #Plastic
    s_4 = kappas[3] #Blood
    KAPPA = MATERIAL(interior,s_1,s_2,s_3, s_4,degree=0)
    
    #Definitio of specific heat
    rhoxcalorespe = [1081.*3686.,1027.*2372.,70.*1045.,1000.*4148.]   
    s_1 = rhoxcalorespe[0] #Atrial wall
    s_2 = rhoxcalorespe[1] #Connective tissue
    s_3 = rhoxcalorespe[2] #Plastic
    s_4 = rhoxcalorespe[3] #Blood
    
    RHOxCALESP = MATERIAL(interior,s_1,s_2,s_3, s_4,degree=0)
    
    #vtkfile_RC = File('salida/rhoCalor.pvd')
    #vtkfile_RC << project_axisym(RHOxCALESP,W_dg_0)    
    
    #Definition of perfusion
    D_prev=interpolate(Constant(0.),W)
    omega=thp.omega
    D_prev_const = project_axisym(D_prev,W_dg_0)# project D onto piecewise constant mesh to stop negative values
    if thp.perf_model=='stop':
        omega=conditional(gt(D_prev_const,thp.p_stop), thp.omega, 0.)
        print("check perfusion threshhold")
        
    
    # Heat transfer variational form
    a = KAPPA*dependenceTkappa*inner(nabla_grad(Theta_), nabla_grad(v))*r*dx+ v*omega*thp.rho*thp.c*Theta_*r*dx+ v*RHOxCALESP/dte*Theta_*r*dx+ht*Theta_*v*r*dss(7)+hp*Theta_*v*r*dss(6)##
    L = f*v*r*dx + v*RHOxCALESP/dte*Theta_prev*r*dx
        
    Theta_ = Function(W)

    # set initial temperature for SAR
    #Theta_.assign(project_axisym(thp.T_initial-310.15,W))
    #Theta_ =  project_axisym(thp.T_initial-310.15,W)

    # assemble in advance of time iteration
    A = None
    b = None

    Q = Function(W_dg)
    
    iu = thp.nu-1
    store_resistance = []#N.array(t_out) # save the resistance at output times
    store_power = []#N.array(t_out) # save power
    store_sensortemp = []# N.array(t_out) # guardar temperatura en sensor
    tiempo = []
    power = 0.
    
    # initialise cell death
    n = len(Theta_.vector().get_local())
    
    
    cda = np.zeros(n) # cell death array
    #cda = cell_death_parameters.A_init
    D = interpolate(Constant(0.),W) # dead field
    
    # control system
    imp_on = True
    imp_off_t_start = dt

    t = dt
    step_inc = True
    
    
    
    #vtkfile = File('data/solution.pvd')
    while t <= tmax+eps:
        # update SAR
        # iu increments each transient iteration, when it reaches nu it updates SAR
        #iu += 1

        #if (iu == thp.nu):
        print("--+--+--             update qRF              --+--+--")
        Q, resistance, power, Vfield = electric_problem(problemname, mesh, interior, boundaries, emp, Theta_prev, thp)
            #iu = 0

        # assemble each iteration to account for previous time step
        dte.dt = dt
        cur_time.t = t

        f.assign(Q)
        
        b = assemble(L, tensor=b)
        A = assemble(a, tensor=A)

        for bc in bcs:
            bc.apply(A, b)
            
            
       

        solve(A, Theta_.vector(), b)# solver_parameters={'linear_solver':'gmres','preconditioner':'ilu'})
        
        sensorTemperature = Theta_(0.001,-0.00116)# Punta del electrodo
  
        #vtkfile << (project_axisym(Theta_+Constant(310.),W), t)

        # adapt T to ensure about 5 time steps per degree change
        nodal_T = Theta_.vector().get_local()
        nodal_T_prev = Theta_prev.vector().get_local()
        T_error = np.abs(nodal_T-nodal_T_prev).max()
        
    
        store_resistance.append(resistance)
        store_power.append(power)
        store_sensortemp.append(sensorTemperature)
        tiempo.append(t)


        # Update cell death
        print("***************************** CELL DEATH  **********************************")
                
        if thp.cda_update:
            cda = cell_death_timestep(cda,n,t,dt,nodal_T,cell_death_parameters)
        
        D.vector()[:] = cda
        D_prev.assign(D) # update cell death for perfusion
        
        Dcellmax = cda.max()
        print("TEM CALCULADA: ",np.abs(nodal_T).max(),"TEMP PREVIA: ",np.abs(nodal_T_prev).max(), "   t: ", t+dt, "   dt: ", dt, "   pow: ", power, "   imp: ", resistance)
        #print("TEM CALCULADA: ",np.abs(nodal_T).max(),"TEMP PREVIA: ",np.abs(nodal_T_prev).max(),"max T err: ", T_error, "   t: ", t+dt, "   dt: ", dt, "   pow: ", power, "   imp: ", resistance,'muerte: ',Dcellmax)
        # ------------------------------
        # Control de temperatura con PI
        # 
        print('TEMPERATURA EN EL SENSOR: ',sensorTemperature)
        ek = Theta_target-sensorTemperature
        xik = xik1+ek #integrador
        
        Pout = Kp*ek+Ki*xik # salida del controlador
        
        if Pout < 0.0:
            Pout = 0.0
        else:
            print('-----------------')
        
        #Tensión del controlador
        Vpi = Pout**0.5
        
        if Vpi > Vmax:
            Vpi = Vmax
            xik = xik1 #no integro si satura
        elif Vpi < Vmin:
            Vpi = Vmin
            xik = xik1 #no integro si satura
        else:
            print('------------------')
            
        xik1 = xik
        
        emp.Vprobe = Vpi
                
        t += dt
        Theta_prev.assign(Theta_)

        step_inc = True # allow increase after accepted step
        print("***************************** ACCEPT TIME **********************************")
        

    #N.savetxt("%s/impedance.out" % problemname, (store_resistance), delimiter=',')   # output impedance

    return project_axisym(Theta_,W), project_axisym(Vfield,W),project_axisym(Theta_+310.15-273.15,W), store_resistance,store_sensortemp,store_power,tiempo


The function to describe the cell death dynamics (solving the ODE).

In [ ]:
def cell_death_timestep(y0,n,t,dt,T_prev_nodal,cdp):
    import scipy.integrate as spint
    step_int = spint.ode(cell_death_func_class)
    step_int.set_integrator("vode",method="bdf",nsteps=1e5)

    step_int.set_f_params([n, cdp.deltaE, cdp.R, cdp.A, T_prev_nodal])
    step_int.set_initial_value(y0,0.)
    step_int.integrate(dt)
    print(step_int.successful())
    if not step_int.successful():
        error("cell death step solve failed")
    cda = step_int.y

    return cda

def project_axisym(func,space):
    r = Expression('x[0]',degree=2)  
    w = TrialFunction(space)
    v = TestFunction(space)
    a = inner(w,v)*r*dx
    L = inner(func, v)*r*dx
    pfunc = Function(space)
    solve(a == L, pfunc)
    return pfunc 

### Scripting

Now, it is time to start to scripting

In [ ]:
EM_parameters.cond = 0.39 #en mm
EM_parameters.V0 = 0.
EM_parameters.Vprobe = 20.0
EM_parameters.cond_model = 'linear'
EM_parameters.restrict_mesh = 1
EM_parameters.cond_rate = 0.015 #1.5%/°C
EM_parameters.cond_vap = EM_parameters.cond*0.01

thermal_parameters.rho_c_t = 1080.*3455. 
thermal_parameters.rho_c_v = 370.*2156. # values for vapourised tissue
thermal_parameters.Lh = 2260.e3 # latent heat of vapourisation
thermal_parameters.Cliq = 0.8 # water content tissue (%)
thermal_parameters.Tu = 373. # upper transition temp
thermal_parameters.Tl = 363. # lower transition temp
thermal_parameters.k = Constant(0.472) 
thermal_parameters.dk = Constant(0.02*0.472)
thermal_parameters.omega = Constant(0.004)
thermal_parameters.rho = Constant(1020.)
thermal_parameters.c = Constant(3400.)
thermal_parameters.T0 = Constant(310.15)
thermal_parameters.T_initial = Constant(310.15)


dt_min = .0001 # absolute step size minimum (0.0001 good), andaba 0.1
dt_max = 1.0 # absolute step size maximum, andaba 1.
tmax = 20.0 # maximum time (s)
dt = .5 # time step (s)
thermal_parameters.perf_model = 'stop'
thermal_parameters.stop_on_me = False

set_log_active(False) # switch off fenics messages

# Load geometry
mesh = Mesh(problemname+".xml")
boundaries = MeshFunction("size_t", mesh, problemname+"_facet_region.xml")
interior = MeshFunction("size_t", mesh, problemname+"_physical_region.xml")


T,V,Tdeg, impedancia1, temperaturaSensor1,potenciaSensor1,tiempo1 = rfa_bioheat_problem(mesh, interior, boundaries, problemname, dt,tmax,thermal_parameters, EM_parameters)

vtkfile_T = File('salida/temperatura.pvd')
vtkfile_T << Tdeg

vtkfile_V = File('salida/tension.pvd')
vtkfile_V << V

print('start time: ', start_time)
print('end time:   ', tm.strftime('%H:%M:%S'))

It is possible to plot the output

In [ ]:
from matplotlib import pyplot as plt

fig1 = plt.figure(1)
f1 = fig1.add_subplot(131)
f1.plot(np.asarray(tiempo1),np.asarray(impedancia1),'.r')
f1 = fig1.add_subplot(132)
f1.plot(np.asarray(tiempo1),np.asarray(temperaturaSensor1)+37.,'.r')
f1.plot(tiempo1,80.*np.ones_like(np.asarray(impedancia1)),'k')
f1 = fig1.add_subplot(133)
f1.plot(np.asarray(tiempo1),np.asarray(potenciaSensor1),'.r')

tiempo = np.asarray(tiempo1)
Pot = np.asarray(potenciaSensor1)
Tem = np.asarray(temperaturaSensor1)
Imp = np.asarray(impedancia1)
np.savetxt('salida/sensorData.out', (tiempo,Pot,Tem,Imp)) 

plt.figure()
p = plot(Tdeg, title="Variación de la temperatura")
plt.colorbar(p)

plt.show()